In [10]:
import os
import yaml
from collections import deque
from typing import(
    Dict,
    List,
    Optional,
    Tuple,
    Union
)

In [4]:
config = "config.default.yml"; config = os.path.abspath(config); config

'/Users/adebayobraimah/Desktop/projects/convert_source/config/config.test/config.default.yml'

In [15]:
def read_config(config_file: Optional[str] = "", 
                verbose: Optional[bool] = False
                ) -> Tuple[Dict[str,str],Dict,Dict,Dict,List[str]]:
    '''
    TODO: 
        * re-write
        
    Reads configuration file and creates a dictionary of search terms for 
    certain modalities provided that BIDS modalities are used as keys. If
    exclusions are provided (via the key 'exclude') then an exclusion list is 
    created. Otherwise, 'exclusion_list' is returned as an empty list. If 
    additional settings are specified, they should be done so via the key
    'metadata' to enable writing of additional metadata. Otherwise, an 
    empty dictionary is returned.

    BIDS modalities:
        - anat:
            - T1w, T2w, FLAIR, etc.
        - func:
            - task:
                - resting state, <task-name>
        - dwi
        - fmap

    Usage example:
        >>> [search_dict, exclusion_list, meta_dict] = read_config(config_file)
    
    Arguments:
        config_file: File path to yaml configuration file. If no file is used, then the default configuration file is used.
        verbose: Prints additional information to screen.
    
    Returns: 
        search_dict: Nested dictionary of heuristic modality search terms for BIDS modalities.
        bids_search: Nested dictionary of heuristic BIDS search terms.
        bids_map: Corresponding nested dictionary of BIDS mapping terms to rename files to.
        meta_dict: Nested dictionary of metadata terms to write to JSON file(s).
        exclusion_list: List of exclusion terms.
    
    Raises:
        ConfigFileReadError: Error that arises if no heuristic search terms are provided.
    '''
    class ConfigFileReadError(Exception):
        pass

    if config_file:
        config_file: str = os.path.abspath(config_file)
    else:
        config_file: str = DEFAULT_CONFIG

    with open(config_file) as file:
        data_map: Dict[str,str] = yaml.safe_load(file)
        if verbose:
            print("Initialized parameters from configuration file")
    
    # Required modality search terms
    if any("modality_search" in data_map for element in data_map):
        if verbose:
            print("Categorizing search terms")
        search_dict: Dict[str,str] = data_map["modality_search"]
        del data_map["modality_search"]
    else:
        if verbose:
            print("Heuristic search terms required. Exiting...")
        raise ConfigFileReadError("Heuristic search terms required. Exiting...")
    
    # BIDS search terms
    if any("bids_search" in data_map for element in data_map):
        if verbose:
            print("Including BIDS related search term settings")
        bids_search: Dict[str,str] = data_map["bids_search"]
        del data_map["bids_search"]
    else:
        if verbose:
            print("No BIDS related search term settings")
        meta_dict: Dict = dict()
    
    # BIDS mapping terms
    if any("bids_map" in data_map for element in data_map):
        if verbose:
            print("Corresponding BIDS mapping settings")
        bids_map: Dict[str,str] = data_map["bids_map"]
        del data_map["bids_map"]
    else:
        if verbose:
            print("No BIDS mapping settings")
        meta_dict: Dict = dict()
    
    # Metadata terms
    if any("metadata" in data_map for element in data_map):
        if verbose:
            print("Including additional settings for metadata")
        meta_dict: Dict[str,Union[str,int]] = data_map["metadata"]
        del data_map["metadata"]
    else:
        if verbose:
            print("No metadata settings")
        meta_dict: Dict = dict()
    
    # Exclusion terms  
    if any("exclude" in data_map for element in data_map):
        if verbose:
            print("Exclusion option implemented")
        exclusion_list: List[str] = data_map["exclude"]
        del data_map["exclude"]
    else:
        if verbose:
            print("Exclusion option not implemented")
        exclusion_list: List = list()
        
    return (search_dict,
            bids_search,
            bids_map,
            meta_dict,
            exclusion_list)

In [36]:
search_dict,bids_search,bids_map,meta_dict,exclusion_list = read_config(config)

In [23]:
search_dict

{'anat': {'T1w': ['T1', 'T1w', 'TFE'], 'T2w': ['T2', 'T2w', 'TSE']},
 'func': {'bold': {'rest': ['rsfMR', 'rest', 'FFE', 'FEEPI']}}}

In [24]:
bids_search

{'anat': {'T1w': {'acq': ['Axial'], 'rec': ['MoCo']},
  'T2w': {'acq': ['Axial'], 'rec': ['MoCo']}},
 'func': {'bold': {'rest': {'acq': ['single'],
    'ce': None,
    'dir': None,
    'rec': None}}}}

In [25]:
bids_map

{'anat': {'T1w': {'acq': ['2D'], 'rec': ['MotionCorrected']},
  'T2w': {'acq': ['2D'], 'rec': ['MotionCorrected']}},
 'func': {'bold': {'rest': {'acq': ['SingleBand'],
    'ce': ['NewContrast'],
    'dir': None}}}}

In [26]:
meta_dict

{'common': {'Manufacturer': 'Philips',
  'ManufacturersModelName': 'Ingenia',
  'MagneticFieldStrength': 3,
  'InstitutionName': "Cincinnati Children's Hospital Medical Center"},
 'func': {'rest': {'ParallelAcquisitionTechnique': 'SENSE',
   'PhaseEncodingDirection': 'j',
   'MultibandAccelerationFactor': 6,
   'TaskName': 'Resting State',
   'dir': 'PA',
   'NumberOfVolumesDiscardedByScanner': 4}}}

In [29]:
exclusion_list

['SURVEY',
 'Reg',
 'SHORT',
 'LONG',
 'MRS',
 'PRESS',
 'DEFAULT',
 'ScreenCapture',
 'PD',
 'ALL',
 'SPECTRO']

In [46]:
def comp_dict(d1: Dict, 
              d2: Dict, 
              path: Optional[str] = "", 
              verbose: bool = False
             ) -> Union[bool,None]:
    '''
    * Use this to compare dictionaries that are supposed to be keymatched *
    
    StackOverflow answer: 
    https://stackoverflow.com/questions/27265939/comparing-python-dictionaries-and-nested-dictionaries
    '''
    for k in d1:
        if (k not in d2):
            if verbose:
                print (path, ":")
                print (k + " as key not in d2", "\n")
            raise KeyError("Input dictionaries do not have matching keys")
        else:
            if type(d1[k]) is dict:
                if path == "":
                    path = k
                else:
                    if verbose:
                        path = path + "->" + k
                    continue
                comp_dict(d1[k],d2[k], path, verbose)
            else:
                if d1[k] != d2[k] and (d1[k] is None or d2[k] is None):
                    if verbose:
                        print (path, ":")
                        print (" - ", k," : ", d1[k])
                        print (" + ", k," : ", d2[k])
                    raise ValueError("One or both input BIDS dictionaries map to NoneType values.")
    return True

In [40]:
comp_dict(bids_search,bids_map,verbose=True)

True

In [43]:
def depth(d: Dict) -> int:
    '''
    StackOverflow answer:
    https://stackoverflow.com/questions/23499017/know-the-depth-of-a-dictionary
    '''
    queue = deque([(id(d), d, 1)])
    memo = set()
    while queue:
        id_, o, level = queue.popleft()
        if id_ in memo:
            continue
        memo.add(id_)
        if isinstance(o, dict):
            queue += ((id(v), v, level + 1) for v in o.values())
    return level

In [44]:
depth(bids_search)

5

In [45]:
depth(bids_map)

5

In [47]:
def myprint(d):
    for k, v in d.items():
        if isinstance(v, dict):
            print(k)
            myprint(v)
        else:
            print(f"{k} : {v}")

In [139]:
search_dict.keys()

dict_keys(['anat', 'func'])

In [150]:
def list_dict(d: Dict[str,str]
             ) -> List[Dict[str,str]]:
    '''
    * This works *
    
    Creates list of dicts
    '''
    arr: List = []
    for k,v in d.items():
        tmp: Dict = {k:v}
        arr.append(tmp)
    return arr

In [142]:
t = list_dict(search_dict)

In [161]:
for i in t:
    for k,v in i.items():
        if depth(i) == 3:
            for k2,v2 in v.items():
                print(f"{k} - {k2} - {v2}")
        elif depth(i) == 4:
            for k2,v2 in v.items():
                for k3,v3 in v2.items():
                    print(f"{k} - {k2} - {k3} - {v3}")

anat - T1w - ['T1', 'T1w', 'TFE']
anat - T2w - ['T2', 'T2w', 'TSE']
func - bold - rest - ['rsfMR', 'rest', 'FFE', 'FEEPI']


In [148]:
depth(t[0])

3

In [303]:
def proc_batch(search_arr):
    '''
    * uses search_arr list
    '''
    for i in search_arr:
        for k,v in i.items():
            if depth(i) == 3:
                for k2,v2 in v.items():
                    modality_type = k
                    modality_label = k2
                    mod_search = v2
                    print(f"{modality_type} - {modality_label} - {mod_search}")
                    # Do stuff here
            elif depth(i) == 4:
                for k2,v2 in v.items():
                    for k3,v3 in v2.items():
                        modality_type = k
                        modality_label = k2
                        task = k2
                        mod_search = v3
                        print(f"{modality_type} - {modality_label} - {task} - {mod_search}")
                        # Do stuff here

In [304]:
proc_batch(t)

anat - T1w - ['T1', 'T1w', 'TFE']
anat - T2w - ['T2', 'T2w', 'TSE']
func - bold - bold - ['rsfMR', 'rest', 'FFE', 'FEEPI']


In [284]:
search_dict['anat']

{'T1w': ['T1', 'T1w', 'TFE'], 'T2w': ['T2', 'T2w', 'TSE']}

In [285]:
t[0]

{'anat': {'T1w': ['T1', 'T1w', 'TFE'], 'T2w': ['T2', 'T2w', 'TSE']}}

In [305]:
bids_search['anat']['T1w'].keys()

dict_keys(['acq', 'rec'])

In [263]:
tt = list_dict(bids_search)

In [273]:
tt[0]["anat"]

{'T1w': {'acq': ['Axial'], 'rec': ['MoCo']},
 'T2w': {'acq': ['Axial'], 'rec': ['MoCo']}}

In [266]:
t[0]

{'anat': {'T1w': ['T1', 'T1w', 'TFE'], 'T2w': ['T2', 'T2w', 'TSE']}}

In [ ]:
def func(L,d=None,mod_t="",mod_l="",task=""):
    if L:
        for i in L:
            if depth(i) == 3:
                
    elif d:
        pass

In [250]:
# Too error prone - abandon
# 
# def organize_terms(search_arr,idx1=0,idx2=0):
#     '''working doc-string'''
#     if depth(search_arr[idx1]) == 3:
#         k1 = list(search_arr[idx1].keys())
#         k2 = list(search_arr[idx1][k1[idx1]])
#         mod_type = k1[idx1]
#         mod_label = k2[idx2]
#         mod_label_search = search_arr[idx1][k1[idx1]][k2[idx2]]
#         return mod_type,mod_label,mod_label_search
#     elif depth(search_arr[idx1]) == 4:
#         k1 = list(search_arr[idx1].keys())
#         k2 = list(search_arr[idx1][k1[idx2]])
#         return k2
# #         for k,v in search_arr[idx1].items():
# #             for k2,v2 in v.items():
# #                 print(f"{k} - {k2} - {v2}")

In [256]:
organize_terms(t,1,0)

['bold']

In [231]:
t[1]

{'func': {'bold': {'rest': ['rsfMR', 'rest', 'FFE', 'FEEPI']}}}

In [146]:
t[1]

{'func': {'bold': {'rest': ['rsfMR', 'rest', 'FFE', 'FEEPI']}}}

In [134]:
t

['anat',
 {'T1w': ['T1', 'T1w', 'TFE']},
 {'T2w': ['T2', 'T2w', 'TSE']},
 'func',
 'bold',
 {'rest': ['rsfMR', 'rest', 'FFE', 'FEEPI']}]

In [136]:
t[1]

{'T1w': ['T1', 'T1w', 'TFE']}